In [1]:
import pandas as pd
import json
import fornax

In [2]:
#populate the database as in tutorial 2
nodes_df = pd.read_csv('./nodes.csv')
edges_df = pd.read_csv('./edges.csv')

In [3]:
nodes_df = pd.read_csv('./nodes.csv')
nodes_df.head()

,label,type,uid
0,Jubilee,0,1126900601
1,Silver Surfer,0,440245546
2,Beast,0,1370118169
3,Selene,0,87770955
4,Sabretooth,0,1299099267


In [4]:
edges_df.head()

,end,start
0,272969114,1126900601
1,741356524,440245546
2,1449626873,1370118169
3,609918602,87770955
4,1449626873,1299099267


In [5]:
query_graph = fornax.GraphHandle.create()
query_graph.add_nodes(id_src=[0, 1, 2], label=['hulk', 'lady', 'storm'])
query_graph.add_edges([0, 1], [1, 2])

In [6]:
target_graph = fornax.GraphHandle.create()
target_graph.add_nodes(id_src=nodes_df['uid'], label=nodes_df['label'], type_=nodes_df['type'])
target_graph.add_edges(edges_df['start'], edges_df['end'])

In [7]:
matches = []
for i in nodes_df[nodes_df['label'].str.contains("(?i)hulk")]['uid']:
    matches.append((0, i, 1))
for i in nodes_df[nodes_df['label'].str.contains('(?i)lady')]['uid']:
    matches.append((1, i, 1))
for i in nodes_df[nodes_df['label'].str.contains('(?i)storm')]['uid']:
    matches.append((2, i, 1))
sources, targets, weights = zip(*matches)

In [8]:
query = fornax.QueryHandle.create(query_graph, target_graph)
query.add_matches(sources, targets, weights)

In [11]:
results = query.execute(n=3)

/Users/dstaff/anaconda3/envs/fornax/lib/python3.6/site-packages/numpy/core/records.py:513: FutureWarning: Numpy has detected that you may be viewing or writing to an array returned by selecting multiple fields in a structured array. 

This code may break in numpy 1.15 because this will return a view instead of a copy -- see release notes for details.
  return obj.view(dtype=(self.dtype.type, obj.dtype))


In [12]:
print(json.dumps(results, indent=4))

{
    "graphs": [
        {
            "cost": 0.024416640711327393,
            "nodes": [
                {
                    "id": 9437002,
                    "type": "query",
                    "id_src": 0,
                    "label": "hulk"
                },
                {
                    "id": 13982314,
                    "type": "query",
                    "id_src": 1,
                    "label": "lady"
                },
                {
                    "id": 76350203,
                    "type": "query",
                    "id_src": 2,
                    "label": "storm"
                },
                {
                    "id": 75367743,
                    "type": "target",
                    "id_src": 37644418,
                    "label": " Susan Storm",
                    "type_": 2
                },
                {
                    "id": 5878004,
                    "type": "target",
                    "id_src": 995920086,
           